# <font color='violet'> Cleaning, Parsing, Feature Engineering on Psychedelic Experience Reports
    
Use the same methods I used for the reviews from studies to clean up the report texts. Address any issues that are unique to the texts from Erowid and require additional cleaning. Then, do feature engineering to create the same columns from data modeled previously: complexity level, similarity with a meta-perfect-review, subjectivity, and polarity. 

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv('../data/raw/erowid/raw_reports_final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8775 entries, 0 to 8774
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8775 non-null   int64 
 1   drug        8775 non-null   object
 2   weight      8775 non-null   object
 3   year        8775 non-null   object
 4   gender      8775 non-null   object
 5   age         8775 non-null   object
 6   report      8775 non-null   object
 7   url         8775 non-null   object
dtypes: int64(1), object(7)
memory usage: 548.6+ KB


In [3]:
df = df.drop(columns=['Unnamed: 0'])
df.columns

Index(['drug', 'weight', 'year', 'gender', 'age', 'report', 'url'], dtype='object')

In [4]:
# Are there rows that are total dupicates of one another?
df = df.drop_duplicates().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5264 entries, 0 to 5263
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   drug    5264 non-null   object
 1   weight  5264 non-null   object
 2   year    5264 non-null   object
 3   gender  5264 non-null   object
 4   age     5264 non-null   object
 5   report  5264 non-null   object
 6   url     5264 non-null   object
dtypes: object(7)
memory usage: 288.0+ KB


In [5]:
# Are there rows that are identical in everything except for the url?
without_urls = df.drop(columns=['url']).drop_duplicates().reset_index(drop=True)
len(without_urls)

5264

In [6]:
# Get my list of target drugs I actually want to analyze. 
drugs_file = open('../data/raw/erowid/psychedelic_drugs.txt', 'r')
drugs_as_string = drugs_file.read()
psychedelic_drugs = drugs_as_string.split(',')
psychedelic_drugs[:10]

['AET',
 'AL-LAD',
 'ALD-52',
 'ALEPH',
 'Aleph-4',
 'Allylescaline',
 'AMT',
 'Arylcyclohexylamines',
 'Ayahuasca',
 'Banisteriopsis caapi']

In [7]:
# Rename the dataframes I'm working with
with_urls = df.copy()

# Drop rows from without_urls for non-target drugs
df = without_urls[without_urls.drug.isin(psychedelic_drugs)].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2948 entries, 0 to 5263
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   drug    2948 non-null   object
 1   weight  2948 non-null   object
 2   year    2948 non-null   object
 3   gender  2948 non-null   object
 4   age     2948 non-null   object
 5   report  2948 non-null   object
dtypes: object(6)
memory usage: 161.2+ KB


In [8]:
# How many reviews are still duplicated?
len(set(df.report))

2397

In [ ]:
# Remove some rows where duplicate reviews are associated with 

<font color='violet'> Next step. 

In [9]:
df.to_csv('../data/processed/erowid_cleaned.csv')